Метод в лоб

In [7]:
import tkinter as tk
from tkinter import ttk
import random

PEAK_HOURS = [(7, 9), (17, 19)]
SHIFT_DURATION = 9
BREAK_DURATION_BIG = 1
BREAK_DURATION_SMALL = 0.25
BREAK_INTERVAL = 3

drivers = [
    {"name": "Водитель 1", "start": 6, "type": "A"},
    {"name": "Водитель 2", "start": 7, "type": "A"},
    {"name": "Водитель 3", "start": 8, "type": "B"},
    {"name": "Водитель 4", "start": 10, "type": "A"},
    {"name": "Водитель 5", "start": 11, "type": "B"},
    {"name": "Водитель 6", "start": 13, "type": "B"},
    {"name": "Водитель 7", "start": 16, "type": "A"},
    {"name": "Водитель 8", "start": 18, "type": "B"},
]

DAYS_OF_WEEK = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]


def is_peak(hour):
    for start, end in PEAK_HOURS:
        if start <= hour < end:
            return True
    return False


def assign_breaks(shift_hours, driver_type):
    breaks = []
    possible_breaks = [hour for hour in shift_hours if not is_peak(hour)]

    if driver_type == "A":
        for i, hour in enumerate(shift_hours):
            if hour in possible_breaks and i >= 4:
                breaks.append(hour)
                break
    elif driver_type == "B":
        first_break = None
        second_break = None
        for i, hour in enumerate(shift_hours):
            if hour in possible_breaks and i >= 2:
                if first_break is None:
                    first_break = hour
                elif (hour - first_break) % 24 >= BREAK_INTERVAL:
                    second_break = hour
                    break
        if first_break is not None:
            breaks.append(first_break)
        if second_break is not None:
            breaks.append(second_break)
    return breaks


def create_driver_schedule(driver):
    schedule = {}
    shift_start = driver['start']
    shift_end = (shift_start + SHIFT_DURATION) % 24
    shift_hours = [(shift_start + i) % 24 for i in range(SHIFT_DURATION)]

    breaks = assign_breaks(shift_hours, driver['type'])

    for hour in shift_hours:
        if hour == shift_start:
            if is_peak(hour):
                schedule[hour] = 'Начало смены (час пик)'
            else:
                schedule[hour] = 'Начало смены'
        elif hour in breaks:
            schedule[hour] = 'Перерыв'
        elif is_peak(hour):
            route_number = random.choice([1, 2])
            schedule[hour] = f'Поездка (час пик, Маршрут №{route_number})'
        else:
            route_number = random.choice([1, 2])
            schedule[hour] = f'Поездка (Маршрут №{route_number})'

    schedule[shift_end] = 'Окончание смены'
    return schedule


def generate_weekly_schedule():
    weekly_schedule = {}
    for day in DAYS_OF_WEEK:
        daily_schedule = []
        for driver in drivers:
            daily_schedule.append(driver)
        weekly_schedule[day] = daily_schedule
    return weekly_schedule


def create_weekly_gui(weekly_schedule):
    root = tk.Tk()
    root.title("Расписание водителей автобусов (Неделя)")
    root.geometry("1800x700")

    canvas = tk.Canvas(root)
    v_scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    h_scrollbar = ttk.Scrollbar(root, orient="horizontal", command=canvas.xview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    v_scrollbar.pack(side="right", fill="y")
    h_scrollbar.pack(side="bottom", fill="x")

    # Функция для сортировки часов с учётом перехода через полночь
    def sort_hours(hours):
        return sorted(hours, key=lambda h: h if h >= 6 else h + 24)

    for day_idx, day in enumerate(DAYS_OF_WEEK):
        day_label = ttk.Label(scrollable_frame, text=f"{day}", font=("Arial", 14, "bold"))
        day_label.grid(row=day_idx * 2, column=0, padx=10, pady=10, sticky="w")

        for driver_idx, driver in enumerate(weekly_schedule[day]):
            driver_schedule = create_driver_schedule(driver)
            driver_type = driver['type']
            frame = ttk.LabelFrame(scrollable_frame, text=f"{driver['name']} (Тип {driver_type})", padding=10)
            frame.grid(row=day_idx * 2 + 1, column=driver_idx, padx=10, pady=5, sticky="n")

            hours_sorted = sort_hours(driver_schedule.keys())
            for hour in hours_sorted:
                activity = driver_schedule[hour % 24]
                label = ttk.Label(frame, text=f"{hour % 24:02d}:00 - {activity}", anchor="w", padding=5)
                label.pack(fill="x")

    root.mainloop()


if __name__ == "__main__":
    weekly_schedule = generate_weekly_schedule()
    create_weekly_gui(weekly_schedule)
